In [15]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import sqlite3
import hvplot.pandas
import datetime as dt

In [16]:
con = sqlite3.connect("covid_database.db")
social_Distancing_df = pd.read_sql_query("SELECT Location, DateTime FROM socialDistancing;", con)
faceMaskViolation_df = pd.read_sql_query("SELECT violatorInfo, Location, DateTime FROM faceMaskViolation;", con)
print("\n\n\n\nacquired data\n\n\n\n")
# Verify that result of SQL query is stored in the dataframe
social_Distancing_df.head()

con.close()






acquired data






In [17]:
social_Distancing_df['DateTime'] = pd.to_datetime(social_Distancing_df['DateTime'])
social_Distancing_df['Date'] = social_Distancing_df['DateTime'].dt.date
social_Distancing_df['Time'] = pd.to_datetime(social_Distancing_df['DateTime']).dt.strftime('%I:%M %p')


faceMaskViolation_df['DateTime'] = pd.to_datetime(faceMaskViolation_df['DateTime'])
faceMaskViolation_df['Date'] = faceMaskViolation_df['DateTime'].dt.date
faceMaskViolation_df['Time'] = pd.to_datetime(faceMaskViolation_df['DateTime']).dt.strftime('%I:%M %p')
faceMaskViolation_df['violatorInfo'] = faceMaskViolation_df['violatorInfo'].astype(str)
del faceMaskViolation_df['DateTime']

In [18]:
faceMaskViolation_df

,violatorInfo,Location,Date,Time
0,102097006,Library,2022-12-18,09:29 AM
1,102097006,COS,2022-12-18,09:30 AM
2,102097006,TAN,2022-12-18,09:30 AM
3,102097006,LP,2022-12-18,09:30 AM
4,102097006,LP,2022-12-18,09:30 AM
5,102097006,TAN,2022-12-18,09:30 AM
6,102097006,LT,2022-12-18,08:01 PM
7,102097006,LP,2022-12-18,08:12 PM
8,102097006,OAT,2022-12-18,08:12 PM


In [19]:
faceMaskViolation_idf = faceMaskViolation_df.interactive()
faceMaskViolation_idf

BokehModel(combine_events=True, render_bundle={'docs_json': {'2d739582-3a68-4dd7-8885-b281c91ced23': {'defs': …

In [20]:
df2 = social_Distancing_df.groupby(['Location', 'Date'])['Location'].count().reset_index(name='counts')
df2['Date'] = pd.to_datetime(df2['Date'])
df2['Date'] = df2['Date'].dt.date
df2.head()

,Location,Date,counts
0,B-Block,2022-12-17,4
1,B-Block,2022-12-18,3
2,COS,2022-12-04,1
3,COS,2022-12-09,1
4,COS,2022-12-17,3


In [21]:
idf = df2.interactive()

In [22]:
social_Distancing_df_table = social_Distancing_df[['Location','Date','Time']]
social_Distancing_idf_table = social_Distancing_df_table.interactive()

In [23]:
min_df = social_Distancing_df["Date"].min()

max_df = social_Distancing_df["Date"].max()

date_slider = pn.widgets.DateSlider(name='Date Slider', start=min_df, end=max_df, value=max_df)
date_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'dc32bf38-f3f8-40ab-b428-49e885bcf886': {'defs': …

In [24]:
locations = ['Library', 'COS', 'Jagii', 'OAT', 'B-Block', 'LT', 'LP', 'TAN']

social_Distancing_pipeline = (
    idf[
        (idf.Date <= date_slider) &
        (idf.Location.isin(locations))
    ]
    .groupby(['Location', 'Date']).mean()
    .reset_index()
    .sort_values(by='Date')  
    .reset_index(drop=True)
)
social_Distancing_table_pipeline = (
    social_Distancing_idf_table[
        (social_Distancing_idf_table.Date <= date_slider) &
        (social_Distancing_idf_table.Location.isin(locations))
    ]
    .reset_index()
    .sort_values(by=['Date','Time'], ascending=False)  
    .reset_index(drop=True)
)



In [34]:
faceMaskViolation_idf_table_pipeline = (
    faceMaskViolation_idf[
        (faceMaskViolation_idf.Date <= date_slider) &
        (faceMaskViolation_idf.Location.isin(locations))
    ]
    .sort_values(by=['Date','Time'], ascending=True)
    .reset_index(drop=True)
)

In [40]:
social_distancing_plot = social_Distancing_pipeline.hvplot(x = 'Date', by='Location', y='counts',line_width=2, title="Social Distancing Violations by Location")
social_distancing_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'d6987218-26a9-4ebe-bc24-2a291ec501cc': {'defs': …

In [41]:
social_Distancing_table = social_Distancing_table_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 7, sizing_mode='stretch_width', show_index = False,disabled = True)
social_Distancing_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'cb8ed426-8d08-4c8e-8f20-84b3ca4ba3da': {'defs': …

In [35]:
faceMaskViolation_idf_table = faceMaskViolation_idf_table_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 7, sizing_mode='stretch_width', show_index = False,disabled = True)
faceMaskViolation_idf_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'ea471887-c30d-4243-acef-8d95c102cb64': {'defs': …

In [30]:
social_distancing_bar = social_Distancing_pipeline.hvplot.bar(stacked=True, x = 'Date', by='Location', y='counts', title="Bar Graph of Social Distancing Violations",legend='top_left')
social_distancing_bar

BokehModel(combine_events=True, render_bundle={'docs_json': {'051ae326-7958-4479-8ef6-63694ddd1297': {'defs': …

## Creating Dashboard

In [18]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Framework for COVID Social Distancing and Mask Detection', 
    sidebar=[pn.pane.Markdown("# COVID-19 Dashboard"), 
             pn.pane.Markdown("#### Management of COVID-19 in the University with the help of Analytics which is powered by AI and ML"), 
             pn.pane.PNG('logo.png', width=300),
             pn.pane.Markdown("## Settings"),   
             date_slider],
    main=[pn.Row(pn.Column(social_distancing_plot.panel(), margin=(0,25)),
                    pn.Column(social_Distancing_table.panel(width=300), margin=(0,25))), 
          pn.Row(pn.Column(social_distancing_bar.panel(), margin=(0,25)),
                    pn.Column(faceMaskViolation_idf_table.panel(width=300), margin=(0,25))
                 )],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();